Análisis de sentimiento. Implementa un módulo de detección de sentimiento de los posts. 

Para entrenar y evaluar diferentes métodos, dispones de la columna “sentiment” en el dataset.  
Al  igual  que  en  el  módulo  de  clasificación  de  subreddits,  Para  evaluar  cada método, se utilizará la métrica f1 score, y se utilizará un 70% de los datos del dataset para entrenamiento y un 30% para test (realizando un sampling aleatorio previo). Deberás probar 
y evaluar los siguientes métodos: 

- Un método basado en lexicons 
- Un método basado en palabras únicas en textos de cada tipo de sentimiento 
- Un método basado en Word embeddings + algoritmo de machine learning de clasificación.

La función de este módulo tendrá como nombre sentiment_analysis(text: str)

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

### Método basado en lexicons

En un método basado en lexicones, no hay un proceso de entrenamiento supervisado convencional; en su lugar, se hace uso de listas o diccionarios de palabras con polaridad (positivas/negativas).

El f1 score se calcula, en este caso, asumiendo un problema binario (positivo/negativo) y se utiliza una división de datos del 70% para entrenamiento y 30% para prueba de manera aleatoria, siguiendo tus requerimientos.

In [ ]:
def sentiment_analysis_lexicons(text: str):
    pass

### Método basado en palabras únicas

In [ ]:
def sentiment_analysis_bow(text:str):
    pass

### Método basado en word embeddgins + ML

Importamos las librerías necesarias, y extraemos unas muestras para entrenamiento y testing para poder ejecutar el código en un tiempo razonable.

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.model_selection import train_test_split
X = reddit_df['clean_post']
y = reddit_df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_emb = X_train.sample(frac=0.1, random_state=42)
y_train_emb = y_train.loc[X_train_emb.index]
X_test_emb = X_test.sample(frac=0.01, random_state=42)
y_test_emb = y_test.loc[X_test_emb.index]

Para la formación de los embeddins usaremos el modelo distilbert, cuya arquitectura deriva del modelo Bert, pero más simplificado, haciéndolo mucho más rápido sacrificando precisión. Usaremos un modelo preentrenado de la librería transformers.

In [ ]:
# Inicializar DistilBERT y el tokenizador
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

Creamos la una clase que herede de las clases BaseEstimator y TransformerMixin de la librería sklearn, para que el modelo sea compatible con los pipeline de sklearn.

In [ ]:
class DistilBERTEmbeddingTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, model_name="distilbert-base-uncased"):
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)

    def fit(self, X, y=None):
        # No se necesita ajuste en este transformador
        return self

    def transform(self, X):
        embeddings = []
        with torch.no_grad():
            for text in X:
                inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True)
                outputs = self.model(**inputs)
                token_embeddings = outputs.last_hidden_state.squeeze(0)
                word_embedding = token_embeddings.mean(dim=0).numpy()
                embeddings.append(word_embedding)
        return embeddings

Entrenamos el modelo.

In [ ]:
embedding_lr_pipeline = Pipeline([('vectorizer',  DistilBERTEmbeddingTransformer()), ('logistic', LogisticRegression())])
embedding_lr_pipeline.fit(X_train_emb, y_train_emb)

Hacemos las predicciones con el conjunte de testeo.

In [ ]:
y_pred_embedding_lr = embedding_lr_pipeline.predict(X_test_emb)

Hacemos las pruebas de f1_score

In [ ]:
f1_score_tfidf_lr = f1_score(y_test_emb, y_pred_embedding_lr, average='weighted')
print(f'F1 score: {f1_score_tfidf_lr}')
print(f'Classification report: {classification_report(y_test, y_pred_tfidf_lr)}')

In [ ]:
print(f'Classification report: {classification_report(y_test, y_pred_tfidf_lr)}')